---
# Solução Exercício Projeto ENEM 2019 v2
---
**Machine Learning em Projetos (Erick Muzart e Fernando Melo)**


## Introdução

### Desempenho prova do ENEM

Modelagem de Desempenho no Enem: identificação das escolas que mais agregam resultados aos seus alunos

Tradicionalmente consideram-se melhores escolas aquelas com melhores resultados médios de seus alunos. No entanto, muito do resultado de um aluno está fortemente relacionado às suas características sócio-econômicas, que também estão associadas à escola que cursa. Deseja-se um método objetivo para distinguir escolas com alunos com desempenhos abaixo ou acima do que se espera deles, em função dessas características socioeconômicas, que possa ser imputado como contribuição da escola.

Por exemplo, se considerarmos apenas o fator renda dos pais, poderia-se agregar as escolas em patamares dessa renda e assim, para cada segmento, identificar as escolas com melhores resultados o que limitaria a influência do fator renda dentro de cada segmento. Idealmente, procuraríamos generalizar esse isolamento de fatores, treinando um modelo preditivo para que estime o desempenho de um aluno em função de todas as suas características, excluindo apenas a escola frequentada. Depois compararíamos esse desempenho estimado com o desempenho real e ordenaríamos as escolas pela contribuição ao desempenho além do esperado.

Os microdados anonimizados do Enem encontram-se publicados pelo Inep (http://inep.gov.br/web/guest/microdados) com boa qualidade de dados em múltiplos anos, para milhões de alunos. Os dados socioeconômicos e de desempenho na prova encontram-se presentes para cada aluno individualmente, viabilizando assim a modelagem desejada. 

Exemplo de análise dos dados do ENEM para identificação de melhores escolas, considerando diversas características: https://exame.com/brasil/7-rankings-mais-realistas-do-desempenho-das-escolas-no-enem/

Micro-dados ENEM 2019: http://download.inep.gov.br/microdados/microdados_enem_2019.zip
650MB compactados, 3GB final

A seguinte análise serve para conhecer melhor os dados disponíveis, apresentar alguma técnicas básicas de análise utilizando pandas e responder a algumas perguntas sobre os dados:

* número total de alunos do Enem 2019:
* relação entre desempenho e 
    - renda dos pais
    - gênero
    - raça
    - tipo de escola, pública ou privada
    - estado de origem
    - etc.

* reproduzir a análise e os resultados do excelente exemplo de jornalismo de dados realizado pelo jornal Estadão: 
http://infograficos.estadao.com.br/educacao/enem/desigualdades-de-genero-e-raca/

Dicionário de dados disponível junto com os dados baixados: recomenda-se a leitura para entendimento do significado das colunas

Aproveitei código de [Gustavo Bonesso](https://www.kaggle.com/gbonesso) para a análise inicial de 2016.

Recomendo começar as análises com um recorte que reduza o número de observações, seja diretamente limitando o número de linhas que serão lidas ou indiretamente selecionando um único estado ou município. O importante é que consiga validar rapidamente suas hipóteses antes de rodar o treinamento com todos os dados disponíveis.

O dataset Enem 2019 contém 5.095.270 de linhas! Então vejam com cuidado qual o tamanho do seu recorte que lhe permite trabalhar com velocidade.

Para facilitar o acesso ao dicionário de dados, fiz um upload separado nos arquivos da turma: https://drive.google.com/file/d/1hUB_2ySBsOfvIQxHxL4cyE7WY_739BhB/view?usp=sharing

### **Descrição do projeto de machine learning**
Antes de escrever qualquer código, precisamos entender o problema que queremos resolver e fazer uma descrição eficiente do projeto, visando a comunicação  simples e rápida do objetivo do projeto para técnicos, gestores e colaboradores.  
  
**1- Descrição do problema ou tarefa:**  
Prever a faixa de renda familiar dos candidatos do ENEM do Distrito Federal com base no questinário sócio-econômico, preenchido no momento da inscrição para a prova.  
**2- Descrição da solução de IA:**  
Treinamento supervisionado de modelo de classificação da renda familiar (17 classes) com os dados das 24 perguntas do questionário sócio-econômico.   
**3- Fonte de dados:**  
Recorte dos Dados do ENEM para o Distrito Federal  
https://drive.google.com/file/d/1hUB_2ySBsOfvIQxHxL4cyE7WY_739BhB/view?usp=sharing  
**4- Variáveis independentes (preditoras ou "features"):**  
'estudo_pai', 'estudo_mae', 'ocupacao_pai', 'ocupacao_mae',
       'num_moradores', 'empregado_domestico', 'banheiro', 'quartos', 'carro',
       'motocicleta', 'geladeira', 'freezer', 'lava_roupa', 'seca_roupa',
       'micro_ondas', 'lava_louca', 'aspirador', 'tv_colorida', 'DVD',
       'tv_assinatura', 'celular', 'tel_fixo', 'computador', 'internet'  
**5- Variável dependente (resposta ou "target"):**  
Renda Familiar (17 classes)    
<br>
-----------------------------------------------------------------
   
Autor do caderno:  Fernando Melo  
Data:  28/10/2021  


## 1 - Carregar os dados e bibliotecas de código

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import os

In [ ]:
# Publiquei o dataset com os dados do Enem 2019 unicamente para o DF, extraído do site do Inep
# DropBox: https://dl.dropboxusercontent.com/s/30z9cido6jykxpf/df_enem2019.csv
!wget https://dl.dropboxusercontent.com/s/30z9cido6jykxpf/df_enem2019.csv


--2021-11-02 16:57:37--  https://dl.dropboxusercontent.com/s/30z9cido6jykxpf/df_enem2019.csv
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.5.15, 2620:100:601d:15::a27d:50f
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.5.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60552861 (58M) [text/csv]
Saving to: ‘df_enem2019.csv’

df_enem2019.csv     100%[===================>]  57.75M  67.5MB/s    in 0.9s    

2021-11-02 16:57:39 (67.5 MB/s) - ‘df_enem2019.csv’ saved [60552861/60552861]



In [ ]:
# Input data files are available in the "./microdados_enem_2019/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
print(os.listdir("."))
path = './df_enem2019.csv'

['.config', 'df_enem2019.csv', 'sample_data']


In [ ]:
# Ler o dataset "df_enem2019.csv" que fizemos o download acima
df = pd.read_csv(path, sep=',')
print('Dimensões do dataset: ', df.shape)
df

Dimensões do dataset:  (94352, 136)


,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,CO_MUNICIPIO_NASCIMENTO,NO_MUNICIPIO_NASCIMENTO,CO_UF_NASCIMENTO,SG_UF_NASCIMENTO,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_ESCOLA,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,IN_BAIXA_VISAO,IN_CEGUEIRA,IN_SURDEZ,IN_DEFICIENCIA_AUDITIVA,IN_SURDO_CEGUEIRA,IN_DEFICIENCIA_FISICA,IN_DEFICIENCIA_MENTAL,IN_DEFICIT_ATENCAO,IN_DISLEXIA,IN_DISCALCULIA,IN_AUTISMO,IN_VISAO_MONOCULAR,...,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001009418,2019,5300108,Brasília,53,DF,20,M,1,1,1,1506138.0,Redenção,15.0,PA,1,2,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,CBCCADEBCEEBECABDCBAEDBDCDBBAAEABBEBBEBAECAAC,BCDDA99999CBDAEEECCCBBE*DEDCBCADEBAABDAAECBCCA...,BCCADEBBABCBADAACEDBDEACBABECEDBEACCECBBAEEAD,0,AAECACDEADCBCDDDBCBDADAEABCEBABEEBCBEECEBDADC,CBABADBBCEEEBCBADCBEEDBBEADBBACDBBACCCCADACAC,BBEDABDACACBABAECBBCCADCEBDBBCDDEEAAADDBECDECA...,EEEADBEBACABCDBABECECACDCBDCCEDCDABEDECDDDBAA,1.0,140.0,120.0,140.0,160.0,140.0,700.0,D,E,C,D,2,G,B,C,D,B,A,B,B,B,A,B,A,A,C,A,A,C,A,B,B
1,190001041173,2019,5300108,Brasília,53,DF,17,F,1,3,1,1302603.0,Manaus,13.0,AM,2,0,2,1.0,0,53000986.0,5300108.0,Brasília,53.0,DF,2.0,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,...,EEBCAEDBADBEABABEBABCCAACDBBEBCCCACBAECBBEABB,EDBCE99999ABBCBDDAAEADDDEAADDCDBBCDCDECEBAECBD...,ECEADDABABCBADEBCEBBDECECEBBCEDCBCACBAABDBCAC,0,AAECACDEADCBCDDDBCBDADAEABCEBABEEBCBEECEBDADC,EEBCEEDBADBBCBABCCADCEBACDBBACCACACBEADBBADCB,ADBBEDCABAABBCBCDAAECDDDBAAAECADECDCEBDEEAECBD...,EEEADBEBACABCDBABECECACDCBDCCEDCDABEDECDDDBAA,1.0,160.0,120.0,140.0,180.0,120.0,720.0,G,G,D,D,2,I,A,B,C,A,A,B,A,B,A,B,A,A,B,A,A,C,B,A,B
2,190001123448,2019,5300108,Brasília,53,DF,22,F,1,3,1,2205805.0,Luzilândia,22.0,PI,1,4,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,CEAACEBABABDAAACEDAECABACDEEEBAABAECDAAEEBDEE,99999BAABEABECABEAAECADAADACBCBBABACEEBEEAEECA...,DDDEBBBBCEBBABECADEDECCCCACAAABCEEDADEBCAEDDE,1,BEEAAEBEEBADEADDADAEABCEDDDBCBCBCCACBCDADCCEB,EEBCEEDBADBBCBABCCADCEBACDBBACCACACBEADBBADCB,ADBBEDCABAABBCBCDAAECDDDBAAAECADECDCEBDEEAECBD...,BEDEEEAADBEBACABCDBABECECACADCBDCCEDCDABECDDD,1.0,120.0,160.0,120.0,160.0,100.0,660.0,F,E,D,B,4,C,A,B,C,A,A,B,A,B,A,A,A,A,B,A,A,D,B,A,B
3,190001128418,2019,5300108,Brasília,53,DF,21,F,1,3,1,3529401.0,Mauá,35.0,SP,1,5,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E,E,A,C,3,B,A,B,C,A,A,B,B,A,A,A,A,A,B,B,B,C,A,A,B
4,190001132590,2019,5300108,Brasília,53,DF,19,F,1,3,1,5300108.0,Brasília,53.0,DF,2,0,2,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E,C,F,F,4,D,C,C,C,B,A,B,A,B,A,B,A,A,C,B,A,C,B,B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94347,190006116422,2019,5300108,Brasília,53,DF,20,F,1,3,1,3520301.0,Iguape,35.0,SP,1,2,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,E,E,F,D,4,F,A,B,B,A,A,B,A,A,A,A,A,A,B,A,A,D,A,A,B
94348,190006116423,2019,5300108,Brasília,53,DF,21,F,1,1,1,5300108.0,Brasília,53.0,DF,1,3,1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,...,EEACEEA

## 2 - Análise exploratória do dados

Selecionar as colunas de interesse para o projeto (feature selection)  
salvar cópia do dataset reduzido para uso futuro  
mostrar as novas dimensões do dataset  
criar gráficos para conhecer melhor os dados de treinamento.

A semântica das variáveis representadas nas colunas do dataset está descrita no dicionário de dados.

Assim, podemos importar seletivamente apenas as colunas que sejam necessárias para a análise.

Exemplo:

### Seleção das variáveis de interesse para o projeto ("feature selection")

In [ ]:
# Seleciona as colunas de interesse
# As colunas do questionário socioeconômico são de nome Q001 a Q025
# A coluna 'Q006' foi colocada no final, pois ela vai ser a variável dependente ("target")
colunas = ['Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025', 'Q006']

In [ ]:
# recorte para o questionário socioeconomico (se)
df_se = df[colunas]
print('Dimensões do novo dataset: ', df_se.shape)
df_se.head()

Dimensões do novo dataset:  (94352, 25)


,Q001,Q002,Q003,Q004,Q005,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q006
0,D,E,C,D,2,B,C,D,B,A,B,B,B,A,B,A,A,C,A,A,C,A,B,B,G
1,G,G,D,D,2,A,B,C,A,A,B,A,B,A,B,A,A,B,A,A,C,B,A,B,I
2,F,E,D,B,4,A,B,C,A,A,B,A,B,A,A,A,A,B,A,A,D,B,A,B,C
3,E,E,A,C,3,A,B,C,A,A,B,B,A,A,A,A,A,B,B,B,C,A,A,B,B
4,E,C,F,F,4,C,C,C,B,A,B,A,B,A,B,A,A,C,B,A,C,B,B,B,D


### Alterar nome das variáveis independentes para facilitar o entendimento

Acesso ao dicionário de dados:  
https://drive.google.com/file/d/1hUB_2ySBsOfvIQxHxL4cyE7WY_739BhB/view?usp=sharing


In [ ]:
# Para tornar mais inteligível, altere o nome das colunas por uma palavra-chave associada à cada questão 
mapa_questao_palavra = {
    'Q001': "estudo_pai",
    'Q002': "estudo_mae",
    'Q003': "ocupacao_pai",
    'Q004': "ocupacao_mae",
    'Q005': "num_moradores",
    'Q006': "renda",
    'Q007': "empregado_domestico",
    'Q008': "banheiro",
    'Q009': "quartos",
    'Q010': "carro",
    'Q011': "motocicleta",
    'Q012': "geladeira",
    'Q013': "freezer",
    'Q014': "lava_roupa",
    'Q015': "seca_roupa",
    'Q016': "micro_ondas",
    'Q017': "lava_louca",
    'Q018': "aspirador",
    'Q019': "tv_colorida",
    'Q020': "DVD",
    'Q021': "tv_assinatura",
    'Q022': "celular",
    'Q023': "tel_fixo",
    'Q024': "computador",
    'Q025': "internet",
}

In [ ]:
# Nome das colunas do dataframe antes
df_se.columns

Index(['Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q007', 'Q008', 'Q009', 'Q010',
       'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016', 'Q017', 'Q018', 'Q019',
       'Q020', 'Q021', 'Q022', 'Q023', 'Q024', 'Q025', 'Q006'],
      dtype='object')

In [ ]:
# Nome das colunas do dataframe depois...
df_se.columns = [mapa_questao_palavra[questao] for questao in df_se.columns]
df_se.columns

Index(['estudo_pai', 'estudo_mae', 'ocupacao_pai', 'ocupacao_mae',
       'num_moradores', 'empregado_domestico', 'banheiro', 'quartos', 'carro',
       'motocicleta', 'geladeira', 'freezer', 'lava_roupa', 'seca_roupa',
       'micro_ondas', 'lava_louca', 'aspirador', 'tv_colorida', 'DVD',
       'tv_assinatura', 'celular', 'tel_fixo', 'computador', 'internet',
       'renda'],
      dtype='object')

In [ ]:
# Mostra as primeiras linhas do dataframe e as novas dimensões 
print('Dimensões do dataset: ',df_se.shape)
df_se.head()

Dimensões do dataset:  (94352, 25)


,estudo_pai,estudo_mae,ocupacao_pai,ocupacao_mae,num_moradores,empregado_domestico,banheiro,quartos,carro,motocicleta,geladeira,freezer,lava_roupa,seca_roupa,micro_ondas,lava_louca,aspirador,tv_colorida,DVD,tv_assinatura,celular,tel_fixo,computador,internet,renda
0,D,E,C,D,2,B,C,D,B,A,B,B,B,A,B,A,A,C,A,A,C,A,B,B,G
1,G,G,D,D,2,A,B,C,A,A,B,A,B,A,B,A,A,B,A,A,C,B,A,B,I
2,F,E,D,B,4,A,B,C,A,A,B,A,B,A,A,A,A,B,A,A,D,B,A,B,C
3,E,E,A,C,3,A,B,C,A,A,B,B,A,A,A,A,A,B,B,B,C,A,A,B,B
4,E,C,F,F,4,C,C,C,B,A,B,A,B,A,B,A,A,C,B,A,C,B,B,B,D


#### O exercício começa aqui
Existem dados nulos?

In [ ]:
# Verificar informações sobre cada uma das colunas do dataframe (.info)


In [ ]:
# Mostra descrição estatítica das variáveis, incluindo as variáveis categóricas (.describe(include='all'))


In [ ]:
# Mostrar gráfico da variável dependente ("target"), com distribuição das amostras de alunos por renda


In [ ]:
# RENDA FAMILIAR, conforme descrito no dicionário de dados:
# A	Nenhuma renda.
# B	Até R$ 998,00.
# C	De R$ 998,01 até R$ 1.497,00.
# D	De R$ 1.497,01 até R$ 1.996,00.
# E	De R$ 1.996,01 até R$ 2.495,00.
# F	De R$ 2.495,01 até R$ 2.994,00.
# G	De R$ 2.994,01 até R$ 3.992,00.
# H	De R$ 3.992,01 até R$ 4.990,00.
# I	De R$ 4.990,01 até R$ 5.988,00.
# J	De R$ 5.988,01 até R$ 6.986,00.
# K	De R$ 6.986,01 até R$ 7.984,00.
# L	De R$ 7.984,01 até R$ 8.982,00.
# M	De R$ 8.982,01 até R$ 9.980,00.
# N	De R$ 9.980,01 até R$ 11.976,00.
# O	De R$ 11.976,01 até R$ 14.970,00.
# P	De R$ 14.970,01 até R$ 19.960,00.
# Q	Mais de R$ 19.960,00.

In [ ]:
# Salvar o dataset resultante da EDA para uso futuro


## 3 -  Pre-processamento dos dados (prepara os dados para treinar o modelo)


In [ ]:
# atribui os valores das variáveis independentes a 'X'
X = df_se.drop(['renda'], axis=1)
# atribui os valores da variável dependente a 'y'
y = df_se['renda']
# Mostra as dimensões de X e y , para conferência
print('Dimensões de X = ', X.shape)
print('Dimensões de y = ', y.shape)

Dimensões de X =  (94352, 24)
Dimensões de y =  (94352,)


### Dividir os dados em treino e teste

In [ ]:
# Partição treino/teste (80/20). Use o parâmetro stratify para garantir a mesma proporção das classes no treino e teste.


In [ ]:
# Mostre as dimensões de treino e teste e confira o resultado.


In [ ]:
# Mostre as dimensões dos dados de teste e confira o resultado.


In [ ]:
# Mostre algumas linhas dos dados de treino e observem que houve um shuffle na ordem das linhas.


In [ ]:
# Mostre o total de amostras por classe dos dados de treino e teste e confira se a proporção está semelhante.


### Codificação das variáveis independentes ("features")
Como temos dados categóricos, precisamos codificá-los numericamente.  
Existem muitas opções, mas como nesse caso os valores das variáveis do ENEM possuem uma ordem inerente, iremos usar o Ordinal Encoder.

In [ ]:
# Importar a biblioteca Ordinal Encoder


In [ ]:
# Instanciar objeto do tipo encoder e ajustar aos dados


In [ ]:
# Ajustar o ordinal encoder aos dados:
# Ajustar o encoder usando todos os dados pode resultar em vazamento de informação (data leakage)
# Portanto, 
# Ajuste o encoder usando somente os dados de treinamento. 


In [ ]:
# Criar um novo dataframe contendo a transformação realizada pelo encoder. 
# A codificação dos dados acontece apenas quando fazemos o .transform()


In [ ]:
# Mostrar as primeiras linhas do dataframe criado acima. Veja que agora só temos numeros.


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,3.0,4.0,2.0,3.0,1.0,1.0,2.0,3.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0
1,6.0,6.0,3.0,3.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0
2,5.0,4.0,3.0,1.0,3.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,1.0
3,4.0,4.0,0.0,2.0,2.0,0.0,1.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,1.0
4,4.0,2.0,5.0,5.0,3.0,2.0,2.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,2.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94347,4.0,4.0,5.0,3.0,3.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,1.0
94348,5.0,4.0,5.0,5.0,3.0,0.0,3.0,3.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,0.0,1.0,4.0,1.0,1.0,1.0
94349,1.0,4.0,2.0,1.0,4.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,1.0
94350,4.0,4.0,3.0,3.0,2.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,1.0


In [ ]:
# Mostre a estatística básica de todas as colunas usando (.describe).
# Verifique se os intervalos de valores das variáveis estão em escalas muito diferentes...


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
count,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000,94352.000000
mean,3.494521,3.536311,2.311928,1.995358,2.853623,0.241871,1.617433,2.362631,0.812373,0.083803,1.061843,0.486837,0.781054,0.148179,0.705973,0.060857,0.257101,1.457754,0.325134,0.342144,2.585955,0.413261,0.988013,0.854110
std,2.035631,1.759560,1.551040,1.469866,1.536467,0.709604,0.915705,0.881673,0.850242,0.306134,0.323960,0.602567,0.453320,0.363973,0.485867,0.246405,0.437038,0.885213,0.468427,0.474430,1.105489,0.492421,0.939182,0.352998
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,1.000000,1.000000,2.000000,0.000000,1.000000,2.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,1.000000
50%,4.000000,4.000000,2.000000,1.000000,3.000000,0.000000,1.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,3.000000,0.000000,1.000000,1.000000
75%,5.000000,5.000000,3.000000,3.000000,4.000000,0.000000,2.000000,3.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,2.000000,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000
max,7.000000,7.000000,5.000000,5.000000,19.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000,4.000000,1.000000,1.000000,4.000000,1.000000,4.000000,1.000000


#### Precisa fazer normalização?
O objetivo da normalização é alterar os valores das colunas numéricas dos dados para uma escala comum, sem distorcer as diferenças nos intervalos de valores.   
Nem sempre precisamos fazer a normalização dos dados.   
É necessário apenas quando as variáveis têm valores com intervalos muito diferentes.  
Caso os valores das variáveis estejam em escalas muito diferentes, tais como, ex: a varável sexo: varie entre 0 e 1, e outra variável salário varie entre 1.000 e 50.000.
      
No nosso caso específico, todas as variáveis estão numa escala entre 0 e 19. Portanto, a princípio não iremos fazer a normalização (StandardScalar no Sklearn).   
Entretanto, se for preciso, pode-se testar e conferir se haverá algum ganho de desempenho no modelo.


## 4 - Treinar o modelo
Modelo básico de regressão logística para pode comparar desempenho com outros modelos a serem treinados no futuro.

In [ ]:
# Instanciar objeto de regressão logística


In [ ]:
# Treinar o modelo, lembrando de aplicar o encoder nos dados de treinamento


In [ ]:
# Verificar a acurácia do modelo (.score) , lembrando de aplicar o encoder nos dados de teste.


In [ ]:
# Verificar o f1_score do modelo. Necessário quando as classes estão desbalenceadas.


## 5 - Avaliar qualidade do modelo

Mas pensem em como vocês fariam para avaliar se o modelo é bom ou não.

Uma boa forma de conferir o desempenho do modelo é fazer uma Matriz de Confusão (Confusion Matrix).   
Desta forma, pode-se verificar o desempenho nas diversas classes.

In [ ]:
# Importar a biblioteca (plot_confusion_matrix)


In [ ]:
# Mostrar a Matriz de Confusão para os diversos patamares de renda familiar


### Conclusão
O modelo errou, praticamente, todas as predições para as classes A, D, F, J, K, L, M.  
Ou seja, ele aprendeu bem apenas as classes B e C, as classes dominantes.  
Será preciso balancear as classes !

## Experimento 2 - Testar balanceamento das 17 classes e treinar novo modelo
Balancear dados  
Codificar numericamente - Ordinal encoder  
Treinar modelo regressão logística  
Avaliar modelo - Confusion Matrix  

In [ ]:
# Se preciso, Ler o dataset df_enem2019_socioecon.csv criado acima.
df_se = pd.read_csv('df_enem2019_socioecon.csv')

### Balanceamento dos dados pelas classes da variável dependente ("target")
Existem várias técnicas, tais como upsampling, downsampling, SMOTE, e outras.

In [ ]:
# Lista a quantidade de amostras por classe (.value_counts)


**Downsampling**   
A classe L é a classe com a menor quantidade de amostras, apenas 1.679 exemplos. Vamos fazer um downsampling, pegando 1679 amostras de cada classe para treinar o modelo inicial.  
Existem várias formas de fazer. Por questões didáticas,  vamos criar um dataframe com os dados de cada uma das 17 classes da variável target. Depois, iremos reduzir estes datasets para apenas 1.679 linhas cada.   
Finalmente, iremos concatenar todos estes datasets, que terá um total de 28.543 linhas (ou seja, 1.679 x 17).

In [ ]:
# import do utilitário resample do sklearn.utils


In [ ]:
# Downsampling todas as classes com exceção da classe L
# Use o comando resample sem permitir reposição (replace=False)

# O dataframe da classe L, classe minoritária, não precisa de resample.

# Concatena todos os dataframes criados

# Mostre as dimensões antes e depois do dwonsampling para conferir o resultado


In [ ]:
# Conferindo o total de linhas desejado para o dataframe resultante do downsampling
1679*17

28543

In [ ]:
# Lista a quantidade de amostras por classe (.value_counts) para verificar o resultado do downsampling


In [ ]:
# atribui os valores das variáveis independentes a 'X'

# atribui os valores da variável dependente a 'y'

# Mostra as novas dimensões de X e y

#### 4 - Treinar o modelo
Treinar novamente a regressão logística para comparar o desempenho, após o balanceamento das classes por subamostragem (downsampling).

In [ ]:
# Modelo básico de regressão logística para pode comparar desempenho com outros modelos a serem treinados no futuro
# Instanciar objeto de regressão logística


In [ ]:
# Treinar o modelo, lembrando de aplicar o encoder nos dados de treinamento


In [ ]:
# Verificar a acurácia do modelo (.score)


In [ ]:
# Verificar o f1_score do modelo, quando as classes estão desbalenceadas


#### 5 - Avaliar qualidade do modelo
Imprimir a matriz de confusão para o novo modelo

In [ ]:
# Matriz de Confusão entre classes de patamares de renda familial


### Conclusão
O modelo melhorou o aprendizado nas diversas classes.  
Entretanto, o desempenho geral do modelo caiu bastante, pois muitos dados foram eliminados com o balanceamento.  
Em função do baixo desempenho, a área de negócio decidiu mudar a definição do projeto:    
O novo objetivo é treinar a predição de 2 classes: 'baixa' e 'alta' renda.   
Para isso, será preciso transformar a variável dependente (target) de forma que:   
'baixa' = classes A, B e C   
'alta' = classes D até Q  


## Experimento 3 - Treinar modelo de classificação binária: renda (baixa/alta)
Ler dados socio-econômicos df_enem2019_socioecon.csv   
Transformar variável target   
pré-processamento   
Treinar modelos   
Avaliar o melhor modelo   


### **Revisão da definição do projeto de machine learning**
Antes de escrever qualquer código, precisamos entender o problema que queremos resolver e fazer uma descrição eficiente do projeto, visando a comunicação  simples e rápida do objetivo do projeto para técnicos, gestores e colaboradores.  
  
**1- Descrição do problema ou tarefa:**  

**2- Descrição da solução de IA:**  
 
**3- Fonte de dados:**  

**4- Variáveis independentes (preditoras ou "features"):**  

**5- Variável dependente (resposta ou "target"):**  


In [ ]:
# Importar bibliotecas (Caso não tenham sido importadas ainda...)


In [ ]:
# Ler o dataset df_enem2019_socioecon.csv


In [ ]:
# Gráfico de barras com total de amostras por nível de renda, seguindo a ordem alfabética das classes: A,B,C...Q


In [ ]:
# Trans formação da variável target de 17 classes para apenas 2 ( 'baixa' ou 'alta')
# baixa = A, B, C
# alta = D até Q
# testa por ordem lexicográfica: A, B e C classe 0, demais letras classe 1

# Mostra as primeiras 20 ocorrências do array 'renda'


In [ ]:
# Atualiza a variável renda (target)  com os novos valores 0=baixa e 1=alta

# Mostra o total de amostras para as novas classes baixa e alta


In [ ]:
# Mostra gráfico de barras comparando o total de amostras para cada classe


In [ ]:
# atribui os valores das variáveis independentes a 'X'

# atribui os valores da variável dependente a 'y'

# Mostra as dimensões de X e y


In [ ]:
# Novo particionamento de dados, dessa vez apenas entre rendas baixa e alta.
# Pelo dicionário de dados, o ponto de corte da renda até a classe C é para valores até ~R$ 1.500

# Partição treino/teste (20%)


In [ ]:
# Como temos dados categóricos, precisamos codificá-los numericamente:
# Instancia e ajusta o ordinal encoder aos dados de treinamento


In [ ]:
%%time
# Modelo básico de regressão logística para pode comparar desempenho:
# Instancie e treine o modelo

# Calcule e mostre a acurácia do modelo 

# Calcule e mostre o f1-score do modelo 


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.11 µs


In [ ]:
# Matriz de Confusão do modelo de renda familiar, acima ou abaixo de R$ 1.500,00 (binário)


In [ ]:
%%time
# Testando outros modelos: Árvore de decisão Classificador (Decision Tree)
# Instancie e treine o modelo

# Calcule e mostre a acurácia do modelo

# Calcule e mostre o f1-score do modelo 


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.58 µs


In [ ]:
%%time
# Testando outros modelos: Floresta Aleatória (Random Forest)
# Instancie e treine o modelo

# Calcule e mostre a acurácia do modelo 

# Calcule e mostre o f1-score do modelo 


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [ ]:
%%time
# (ATENÇÃO !!! tempo estimado: 7 minutos )
# Testando outros modelos: SVM (Suport Vector Machine)
# Instancie e treine o modelo

# Calcule e mostre a acurácia do modelo 

# Calcule e mostre o f1-score do modelo 


CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.11 µs


In [ ]:
# Matriz de Confusão do melhor modelo (SVM)


Reagrupando os resultados colhidos dos modelos testados:

| modelo | acurácia | F1 score | variável | tempo processamento  
|--------|----------|----------|----|----|
| DecisionTreeClassifier | --- | --- | tree | ---
| RandomForestClassifier | --- | --- | rf | ---
|LogisticRegression | --- | --- | log_reg |---|  
| SVM | --- | --- | svm_clf | --- |  


#### Testa acurácia dos modelos com Cross-Validation cria ensemble
Para uma avaliação melhor do desempenho dos modelos treinados, iremos verificar a acurácia dos modelos com cross-validation.   
Para fins didáticos , iremos usar k-fold=5.

In [ ]:
%%time
# (tempo estimado: 2 minutos)
# Cria o ensemble (VotingClassifier) dos modelos treinados (sem o SVM )

# Compara diversos modelos treinados com o ensemble
# Mostrar a acurácia de cada modelo (tree, RF, RL e ensemble) com cross-validation (imprimir média e desvio padrão)


Decision Tree : 0.719 (+/- 0.012)
Random Forest : 0.785 (+/- 0.012)
Logistic Regression : 0.790 (+/- 0.014)
Ensemble : 0.786 (+/- 0.013)


CPU times: user 1min 54s, sys: 11.6 s, total: 2min 5s
Wall time: 1min 51s


In [ ]:
%%time
# (ATENÇÃO !!! tempo estimado: 58 minutos)
# Cria o ensemble dos modelos treinados (incluindo o SVM (muito demorado!))
# eclf = VotingClassifier(
#     estimators=[('tree', tree), ('RF', rf), ('log_reg', log_reg), ('SVM', svm_clf)], voting='hard')
# # Compara diversos modelos treinados com o ensemble
# for clf, label in zip([tree, rf, log_reg, svm_clf, eclf], [ 'Decision Tree', 'Random Forest', 'Logistic Regression', 'SVM', 'Ensemble']):
#     scores = cross_val_score(clf, enc.transform(X), y, scoring='accuracy', cv=5) # specify the number of folds in a (Stratified)KFold
#     print(f"{label} : {scores.mean():0.3f} (+/- {scores.std():0.3f})")
# print('\n')

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


## Que modelo você colocaria em produção, caso tivesse que usar os dados de todo o Brasil?

In [ ]:
# Tempo de treinamento do modelo de SVM em horas
(395*5095000)/94352/60/60

5.924990696776138

In [ ]:
# Tempo de treinamento do modelo de Regressão logistica em minutos
(1.7*5095000)/94352/60

1.5299975976485218

### Tempo aproximado para treinamento dos modelos:
SVM = 6 horas.  
RL = 2 minutos

## Próximos passos
Investigar:  
- One-hot encoding
- normalização Standard Scaler
- Selecionar Variáveis independentes com feature importance por permutação  



## Importância das variáveis por permutação (feature importance permutation)
A importância das variáveis por permutação é definida como a diminuição na performance de um modelo quando os valores de uma das features é embaralhado aleatoriamente.   
Este procedimento quebra a relação entre a feature e a variável target, portanto, a queda na performance do modelo é indicativa de quanto o modelo depende desta feature. Esta técnica pode ser usada com qualquer modelo e pode ser calculada muitas vezes com diferentes permutações das features.   
fonte(https://scikit-learn.org/stable/modules/permutation_importance.html)


In [ ]:
# Calcula a importância das variáveis 
# Import a biblioteca (permutation_importance)

# Calcular os resultados (permutation_importance) da queda no desempenho do modelo 
#          ao embaralhar cada uma das variáveis.

# Ordenar os resultados

# Gráfico boxplot pra mostrar a importância das variáveis






In [ ]:
# Mostra o nome das colunas dos dados de teste em ordem de importância


In [ ]:
# atribui os valores das variáveis independentes a 'X', eleiminando as 7 menos importantes

# atribui os valores da variável dependente a 'y'

# Mostra as novas dimensões de X e y


In [ ]:
# Novo particionamento de dados
# Partição treino/teste (80/20)


In [ ]:
# Como temos dados categóricos, precisamos codificá-los numericamente:
# Instanciar e codificar os dados das variáveis de treinamento


In [ ]:
# Instancie e treine o modelo

# Calcule e mostre a acurácia do modelo 

# Calcule e mostre o f1-score do modelo 


## Plotar uma árvore decisão para entender as variáveis que melhor explicam o fenômeno subjacente
Criar árvore de decisão com os dados de treino, limitando a profundidade, de forma que fique legível.   
Mostrar as descobertas para o especialista de negócio e validá-las.  
Se um novo candidato do Enem não quisesse preencher o questionário sócio econômico, mas aceitasse responder apenas uma pergunta, qual seria a pergunta mais importante, com a qual poderiamos fazer uma estimativa mais segura se o candidato tem renda familiar baixa ou alta ?

In [ ]:
# Importar bibliotecas (matplotlib e plot_tree)


In [ ]:
# Plotar a árvore de decisão
# ajuste o tamanho da figura para facilitar a visualização

# Mostrar a árvore passando os dados de treinamento e profundidade máxima = 4


### Aplicamos diversas técnicas sobre o dataset ENEM:

* Pré-processamento: 
    * encoding de dados categóricos
    * imputação de dados faltantes
    * seleção de features
* Particionamento treino/validação
* Modelos simples: avaliação, casos de erro
* Cross validation...
* diversos modelos: Logistic Regression, SVM, Decision Tree, Random Forest
* gráficos e interpretação de árvores de decisão
* Ensemble de modelos